## Gerekli Kütüphaneler

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

## Veri Setinin Eklenmesi

In [2]:
df = pd.read_csv('Restaurant_Reviews_Cleaned.csv')
df = df.dropna().reset_index(drop=True)
print(df.info())
df.head(167)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Review  700 non-null    object 
 1   Liked   700 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.1+ KB
None


,Review,Liked
0,Wow... Loved this place.,1.0
1,Crust is not good.,0.0
2,Not tasty and the texture was just nasty.,0.0
3,Stopped by during the late May bank holiday of...,1.0
4,The selection on the menu was great and so wer...,1.0
...,...,...
162,If she had not rolled the eyes we may have sta...,0.0
163,Horrible - don't waste your time and money.,0.0
164,Now this dish was quite flavourful.,1.0
165,By this time our side of the restaurant was al...,0.0


## Veri Ön İşleme

In [3]:
#Veri setindeki noktalama işaretleri metni işleme açısından sıkıntı yaratıyor.
#Çünkü play kelimesiyle play... aynı şeye hitap etse de makine bunu aynı kelime olarak algılamıyor.
#Bu sebeple metinlerdeki noktalama işaretlerini çıkarmamız gerekiyor.
#Aynı zamanda metindeki Play ile play aynı anlama gelmesine rağmen makine aynı olarak değerlendiremeyeceği için bütün harfleri küçültmemiz gerekiyor.
#İngilizcede that did gibi (bu operasyon için) kelimeler gereksiz olacağından stopwords kullanarak metinden atıyoruz.
#ps.steam kullanarak kelimelerin sadece gövdesini alıyoruz. (played --> play) gibi...

In [4]:
import re #Regular Expression

import nltk #Natural Language Toolkit
stopWords = nltk.download('stopwords')

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

from nltk.corpus import stopwords

derlem = []
for i in range(700):
    satir = re.sub('[^a-zA-Z]',' ',df['Review'][i])
    satir = satir.lower()
    satir = satir.split()
    satir = [ps.stem(kelime) for kelime in satir if not kelime in set(stopwords.words('english'))]
    satir = ' '.join(satir)
    derlem.append(satir)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kirsa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [5]:
from sklearn.feature_extraction.text import CountVectorizer #Feature Extraction (Öznitelik Çıkarımı)
cv = CountVectorizer(max_features=2000)
x = cv.fit_transform(derlem).toarray()
y = df.iloc[:,1]

In [6]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.20,random_state=0)

## Modelin Oluşturulması

In [7]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(xtrain,ytrain)
ypred = gnb.predict(xtest)

## Model Değerlendirmesi

In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest,ypred)
print(cm) #140 veriden 100'ü doğru %71.40 Accuracy oranı var.

[[38 30]
 [10 62]]
